##### Step 1: Creating pose detection using MediaPipe Pose Landmarks

Reference: 
- https://colab.research.google.com/github/googlesamples/mediapipe/blob/main/examples/pose_landmarker/python/%5BMediaPipe_Python_Tasks%5D_Pose_Landmarker.ipynb
- https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker

In [1]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [135]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import cv2

base_options = python.BaseOptions(model_asset_path='pose_detection_mediapipe/pose_landmarker_heavy.task')
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

image = mp.Image.create_from_file("Dataset/full-lunges/4.jpg")

detection_result = detector.detect(image)

annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)
cv2.imshow("Show image", cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

##### Step 2: Calculating the angle between the ankle, knee, hip, and shoulders
These are the angles that we will be calculating for detecting the lunges

In [70]:
import math

def calculate_angle(landmark1, landmark2, landmark3):
    x1, y1, _ = landmark1.x, landmark1.y, landmark1.z
    x2, y2, _ = landmark2.x, landmark2.y, landmark2.z
    x3, y3, _ = landmark3.x, landmark3.y, landmark3.z

    angle = math.degrees(math.atan2(y3 - y2, x3 - x2) - math.atan2(y1 - y2, x1 - x2))

    if angle < 0:
        angle += 360

    if angle > 180:
        angle = 360 - angle

    return angle

In [133]:
from mediapipe import solutions
mp_pose = solutions.pose
def classify_pose(landmarks):
    # Get the landmarks
    # Right shoulder, right hip, right knee
    label = "Unknown Pose"
    left_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE])
    
    print(f"Left Knee Angle: {left_knee_angle}")

    right_knee_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP], 
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE], 
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE])
    print(f"Right Knee Angle: {right_knee_angle}")

    if (left_knee_angle > 75 and left_knee_angle < 115) and (right_knee_angle > 75 and right_knee_angle < 115):
        # Both knees are bent -> can be a squat
        left_waist_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_HIP], 
                                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE])
        right_waist_angle = calculate_angle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER],
                                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP],
                                        landmarks[mp_pose.PoseLandmark.RIGHT_KNEE])
        
        print(f"Left Waist Angle: {left_waist_angle}")
        print(f"Right Waist Angle: {right_waist_angle}")
        # A full Lunge must have one of the waist angles straight
        if (left_waist_angle > 160 and left_waist_angle < 180) or (right_waist_angle > 160 and right_waist_angle < 180):
            return "Full Lunge"

        return "Squat"
    if (left_knee_angle > 120 and left_knee_angle < 180) and (right_knee_angle >= 115 and right_knee_angle < 145):
        # Right knee is bent
        return "Half Lunge (R)"
    if (left_knee_angle >= 115 and left_knee_angle < 145) and (right_knee_angle > 120 and left_knee_angle < 180):
        # Left knee is bent
        return "Half Lunge (L)"
    return label

In [136]:
classify_pose(detection_result.pose_landmarks[0])

Left Knee Angle: 111.89136418751362
Right Knee Angle: 106.13665765218714
Left Waist Angle: 167.17573581496387
Right Waist Angle: 114.34267245883012


'Full Lunge'